In [1]:
!pip install tensorflow tensorflow_hub tensorflow_datasets

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds


In [3]:
def get_data():
    datasets = tfds.load(name='mnist', as_supervised=True)
    mnist_train, mnist_test = datasets['train'], datasets['test']

    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def scale(image, label):
        image = tf.cast(image, tf.float32) / 255.0
        return image, label

    train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

    return train_dataset, eval_dataset

train_dataset, eval_dataset = get_data()


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

model = get_model()
model.fit(train_dataset, epochs=2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 37s 35ms/step - accuracy: 0.8811 - loss: 0.4098
Epoch 2/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 30ms/step - accuracy: 0.9768 - loss: 0.0781


In [5]:
keras_model_path = '/content/keras_model.keras'
model.save(keras_model_path)


In [6]:
restored_model = tf.keras.models.load_model(keras_model_path)
restored_model.fit(train_dataset, epochs=2)  # Continue training


Epoch 1/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 34s 35ms/step - accuracy: 0.9846 - loss: 0.0533
Epoch 2/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 37s 30ms/step - accuracy: 0.9884 - loss: 0.0402


In [ ]:
saved_model_path = "/content/saved_model"
tf.saved_model.save(model, saved_model_path)


In [10]:
tf.saved_model.save(model, saved_model_path)


In [11]:
saved_model_path = "/content/saved_model"  # Define the path again
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures["serving_default"]

for image, label in eval_dataset.take(1):
    print(inference_func(image))


{'output_0': <tf.Tensor: shape=(64, 10), dtype=float32, numpy=
array([[ -2.417013  ,  -1.0697418 ,  16.095827  ,   1.8225973 ,
          1.7944095 ,  -1.5048654 ,  -4.393439  ,  -1.9254258 ,
          4.880852  ,   2.1309254 ],
       [ 13.394815  ,  -6.7701907 ,   0.8087504 ,  -7.336992  ,
        -11.771863  ,   4.139698  ,   2.427309  ,  -5.1533046 ,
          4.426559  ,  -1.3959538 ],
       [ -2.4256687 ,  -3.0347154 ,  -1.6079118 ,  -1.2845442 ,
         12.9567    ,  -1.1781021 ,  -2.6209297 ,   4.776591  ,
          1.6451836 ,   4.824807  ],
       [ -2.4310584 ,  -3.980094  ,   2.0302224 ,   3.407247  ,
         -4.788639  ,  -0.52263224,  -0.968201  ,  -2.5918822 ,
         12.541449  ,  -2.4557989 ],
       [ -0.17491001,  -3.0566306 ,   3.2995272 ,   1.1517582 ,
         -5.9395404 ,  -0.6242918 ,  -9.245918  ,  13.989462  ,
          1.6769426 ,   2.3984108 ],
       [ -0.4661992 ,  -7.2861886 ,  -1.1645077 ,  -3.4807546 ,
         -3.107708  ,   3.1324415 ,  13.423988  